## Clowder APIs

## Import libraries and setup utility function

We start by importing the rquired libraries, data and setting up some utility functions and variables that we will use below.

In [43]:
import pyclowder
import json
import os
import pandas as pd

import requests

# Function to download the IRIS dataset
def download_iris_dataset():
    # URL for the Iris dataset hosted by UCI Machine Learning Repository
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.content
    else:
        print("Failed to download the dataset. Status code:", response.status_code)


CLOWDER_URL = "http://localhost:8000"

## Token Generation

In [44]:
user_login_json = {
    "email": "mohanar2@illinois.edu",
    "password": "password"
}
login_url = CLOWDER_URL + "/api/v2/login"
response = requests.post(login_url, json = user_login_json)
token = response.json()["token"]
headers = {
    'Authorization': f'Bearer {token}'
}
print(response.json())

{'token': 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICI2dUVlQ0xOc1hTQXZUN1VDek1FRVk2VmI4ajJnY1RhWlFESUpnbnFGSHVJIn0.eyJleHAiOjE3MTk5MzAzNTMsImlhdCI6MTcxOTkzMDA1MywianRpIjoiYjZhYTAwNjUtZmY1OS00M2QyLWJiOGYtNzliNGI0MTgwNzNmIiwiaXNzIjoiaHR0cDovL2xvY2FsaG9zdDo4MDgwL2tleWNsb2FrL3JlYWxtcy9jbG93ZGVyIiwic3ViIjoiZjg0Y2JjNmQtYzEzZC00MmVmLWFhN2MtMWQ4MmFjYzVhZWViIiwidHlwIjoiQmVhcmVyIiwiYXpwIjoiY2xvd2RlcjItYmFja2VuZCIsInNlc3Npb25fc3RhdGUiOiJlZDM0MzA3Ny1lZmNiLTRlOGMtYjM4OS0zN2JlMzk0MmUyNzAiLCJhbGxvd2VkLW9yaWdpbnMiOlsiaHR0cDovL2xvY2FsaG9zdDo4MDAwIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJkZWZhdWx0LXJvbGVzLWNsb3dkZXIiLCJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIl19LCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGVtYWlsIiwic2lkIjoiZWQzNDMwNzctZWZjYi00ZThjLWIzODktMzdiZTM5NDJlMjcwIiwiZW1haWxfdmVyaWZpZWQiOnRydWUsIm5hbWUiOiJWaXNtYXlhayBNb2hhbmFyYWphbiIsInByZWZlcnJlZF91c2VybmFtZSI6Im1vaGFuYXIyQGlsbGlub2lzLmVkdSIsImdpdmVuX25hbWUiOiJWaXNtYXlhayIsImZhbWlseV9uYW1lIjoiTW9oYW5hcmFqYW4iLCJlbWFpbCI6Im1vaGFuYXIyQGlsbGlub2lzL

## Creating Dataset

In [45]:
dataset_json = {
    "name": "Flower Dataset",
    "description": "Dataset for Flower Data",
    "status": "PRIVATE",
}
dataset_params = {
    "license_id": "CC BY"
}


dataset_url = CLOWDER_URL + "/api/v2/datasets"
response = requests.post(dataset_url, json = dataset_json, headers = headers, params = dataset_params)
dataset_id = response.json()['id']
print(response.json())

{'name': 'Flower Dataset', 'description': 'Dataset for Flower Data', 'status': 'PRIVATE', 'id': '66840cc55713c91cd9b89483', 'creator': {'email': 'mohanar2@illinois.edu', 'first_name': 'Vismayak', 'last_name': 'Mohanarajan', 'id': '663a45d5b75ca83d17ac6564', 'admin': True, 'admin_mode': True, 'read_only_user': False}, 'created': '2024-07-02T14:20:53.503934', 'modified': '2024-07-02T14:20:53.503942', 'user_views': 0, 'downloads': 0, 'thumbnail_id': None, 'standard_license': True, 'license_id': 'CC BY'}


## Uploading File

In [47]:

file = download_iris_dataset()
file_json = {
    "file": file,
    "mediaType": 'multipart/form-data'
}
save_file_url = CLOWDER_URL + "/api/v2/datasets/" + dataset_id + '/files'
headers = {
    'Authorization': f'Bearer {token}'
}
response = requests.post(save_file_url, files = file_json, headers = headers)
print(response.json())

{'name': 'file', 'status': 'PRIVATE', 'id': '66840ced5713c91cd9b89485', 'creator': {'email': 'mohanar2@illinois.edu', 'first_name': 'Vismayak', 'last_name': 'Mohanarajan', 'id': '663a45d5b75ca83d17ac6564', 'admin': True, 'admin_mode': True, 'read_only_user': False}, 'created': '2024-07-02T14:21:33.733027', 'version_id': '2a4bbb53-8149-4384-9d55-2696c9d587ed', 'version_num': 1, 'dataset_id': '66840cc55713c91cd9b89483', 'folder_id': None, 'views': 0, 'downloads': 0, 'bytes': 4551, 'content_type': {'content_type': 'application/octet-stream', 'main_type': 'application'}, 'thumbnail_id': None, 'storage_type': 'minio', 'storage_path': None, 'object_type': 'file'}
